In [1]:
#import required libraries
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
import cv2
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, precision_score, f1_score, recall_score
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, SimpleRNN,Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd
import time
import psutil
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
import keras

tf.keras.utils.set_random_seed(0)
tf.config.experimental.enable_op_determinism()

2024-03-25 11:07:21.174899: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 11:07:21.174971: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 11:07:21.177654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 11:07:21.195648: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 11:07:23.789884: W ten

In [2]:
#!pip install tensorflow-tensorrt


In [3]:
#!pip install opencv-python
#!pip install imutils
#!pip install scikit-learn
#!pip install xgboost


In [4]:
#Class to load the dataset images from device

class SimpleDatasetLoader:
    # Method: Constructor
    def __init__(self, preprocessors=None):
        """
        :param preprocessors: List of image preprocessors
        """
        self.preprocessors = preprocessors

        if self.preprocessors is None:
            self.preprocessors = []

    # Method: Used to load a list of images for pre-processing
    def load(self, image_paths, verbose=-1):
        """
        :param image_paths: List of image paths
        :param verbose: Parameter for printing information to console
        :return: Tuple of data and labels
        """
        data, labels = [], []

        for i, image_path in enumerate(image_paths):
            image = cv2.imread(image_path)
            label = image_path.split(os.path.sep)[-2]

            if self.preprocessors is not None:
                for p in self.preprocessors:
                    image = p.preprocess(image)

            data.append(image)
            labels.append(label)

            if verbose > 0 and i > 0 and (i+1) % verbose == 0:
                print('[INFO]: Processed {}/{}'.format(i+1, len(image_paths)))

        return (np.array(data), np.array(labels))

In [5]:
#Class Preprocessror 
class SimplePreprocessor:
    # Method: Constructor
    def __init__(self, width, height, interpolation=cv2.INTER_AREA):
        """
        :param width: Image width
        :param height: Image height
        :param interpolation: Interpolation algorithm
        """
        self.width = width
        self.height = height
        self.interpolation = interpolation

    # Method: Used to resize the image to a fixed size (ignoring the aspect ratio)
    def preprocess(self, image):
        """
        :param image: Image
        :return: Re-sized image
        """
        return cv2.resize(image, (self.width, self.height), interpolation=self.interpolation)

In [6]:
from imutils import paths
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from __main__ import SimplePreprocessor
from __main__ import SimpleDatasetLoader

In [7]:
# Function to load and preprocess data using SimpleDatasetLoader
def load_and_preprocess_data(image_paths, target_size):
    sp = SimplePreprocessor(target_size[0], target_size[1])
    sdl = SimpleDatasetLoader(preprocessors=[sp])
    data, labels = sdl.load(image_paths, verbose=1000)

    print(data.shape)
    print(labels.shape)

    # Convert labels to one-hot encoding
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    #labels = to_categorical(labels)

    return data, labels



# Get list of image paths
image_paths = list(paths.list_images("../cell_images/"))

# Define target size for images
target_size = (224, 224)  # Change this to your desired size

# Load and preprocess data
print('[INFO]: Images loading....')
data, labels = load_and_preprocess_data(image_paths, target_size)

[INFO]: Images loading....
[INFO]: Processed 1000/27558
[INFO]: Processed 2000/27558
[INFO]: Processed 3000/27558
[INFO]: Processed 4000/27558
[INFO]: Processed 5000/27558
[INFO]: Processed 6000/27558
[INFO]: Processed 7000/27558
[INFO]: Processed 8000/27558
[INFO]: Processed 9000/27558
[INFO]: Processed 10000/27558
[INFO]: Processed 11000/27558
[INFO]: Processed 12000/27558
[INFO]: Processed 13000/27558
[INFO]: Processed 14000/27558
[INFO]: Processed 15000/27558
[INFO]: Processed 16000/27558
[INFO]: Processed 17000/27558
[INFO]: Processed 18000/27558
[INFO]: Processed 19000/27558
[INFO]: Processed 20000/27558
[INFO]: Processed 21000/27558
[INFO]: Processed 22000/27558
[INFO]: Processed 23000/27558
[INFO]: Processed 24000/27558
[INFO]: Processed 25000/27558
[INFO]: Processed 26000/27558
[INFO]: Processed 27000/27558
(27558, 224, 224, 3)
(27558,)


In [8]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(target_size[0], target_size[1], 3))
model = Model(inputs=base_model.input, outputs=base_model.output)
# Extract features
features = model.predict(data, batch_size=32, verbose=1)
 
# Flatten the features
features_flatten = features.reshape(features.shape[0], -1)
    
print(features_flatten.shape)

2024-03-25 11:08:26.843238: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 37894 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:65:00.0, compute capability: 8.0
2024-03-25 11:08:52.166059: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-25 11:08:52.763819: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


862/862 [==============================] - 23s 19ms/step
(27558, 25088)


In [9]:
data = features_flatten

In [13]:
import gc
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
#tf.config.experimental.enable_tensor_float_32_execution(True)
#tf.config.threading.set_inter_op_parallelism_threads(1)
#tf.config.threading.set_intra_op_parallelism_threads(1)
# Initialize a DataFrame to store results
results_df = pd.DataFrame(columns=['Model', 'Average Accuracy', 'Average Sensitivity', 'Average Specificity',
                                   'Average AUC-ROC', 'Average MCC', 'Average Precision', 'Average F1 Score',
                                   'Memory Used (MB)', 'Time (s)'])

# Function to evaluate a model
def evaluate_model(model, name, data, labels):
    if name == "VGG16":
        labels = to_categorical(labels)

    elif name == "VGG19":  
        labels = to_categorical(labels)
    else:
        data = data
    accuracy_list = []
    sensitivity_list = []
    specificity_list = []
    auc_roc_list = []
    mcc_list = []
    precision_list = []
    f1_list = []
    time_start = time.time()
    memory_start = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    if name == "VGG16" or name == "VGG19":


        with tf.device('/GPU:0'):
            tf.random.set_seed(0)
            for fold, (train_index, test_index) in enumerate(skf.split(data, labels.argmax(axis=1))):
                print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

                X_train, X_test = data[train_index], data[test_index]
                y_train, y_test = labels[train_index], labels[test_index]


                # Adding early stopping to prevent overfitting
                early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
                keras.utils.set_random_seed(0)
                np.random.seed(0)
                tf.random.set_seed(0)
                #tf.config.run_functions_eagerly(True)
                tf.data.experimental.enable_debug_mode()
                # Train the model
                # Example of using tf.data.Dataset
                train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
                train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(64)

                model.fit(train_dataset, epochs=20, callbacks=[early_stopping])

                # Evaluate the model
                predictions = model.predict(X_test)
                y_pred = np.argmax(predictions, axis=1)
                y_test_encoded = np.argmax(y_test, axis=1)

                
                # Calculate evaluation metrics for the current fold
                accuracy = np.mean(y_pred == y_test_encoded)
                sensitivity = recall_score(y_test_encoded, y_pred, pos_label=1)
                specificity = recall_score(y_test_encoded, y_pred, pos_label=0)
                auc_roc = roc_auc_score(y_test_encoded, y_pred)
                mcc = matthews_corrcoef(y_test_encoded, y_pred)
                precision = precision_score(y_test_encoded, y_pred, pos_label=1)
                f1 = f1_score(y_test_encoded, y_pred, pos_label=1)
                # Append metrics to lists
                accuracy_list.append(accuracy)
                sensitivity_list.append(sensitivity)
                specificity_list.append(specificity)
                auc_roc_list.append(auc_roc)
                mcc_list.append(mcc)
                precision_list.append(precision)
                f1_list.append(f1)

                # Print metrics for the current fold
                print(f"Accuracy: {accuracy}")
                print(f"Sensitivity: {sensitivity}")
                print(f"Specificity: {specificity}")
                print(f"AUC-ROC: {auc_roc}")
                print(f"MCC: {mcc}")
                print(f"Precision: {precision}")
                print(f"F1 Score: {f1}")
                gc.collect()

            # Calculate average metrics
            average_accuracy = np.mean(accuracy_list)
            average_sensitivity = np.mean(sensitivity_list)
            average_specificity = np.mean(specificity_list)
            average_auc_roc = np.mean(auc_roc_list)
            average_mcc = np.mean(mcc_list)
            average_precision = np.mean(precision_list)
            average_f1 = np.mean(f1_list)

            time_end = time.time()
            memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
            del X_train, X_test, y_train, y_test
            gc.collect()
            tf.keras.backend.clear_session()
    else:
        for fold, (train_index, test_index) in enumerate(skf.split(data, labels)):
            print(f'\n[INFO] Fold {fold + 1} / 10 for {name}')

            X_train, X_test = data[train_index], data[test_index]
            y_train, y_test = labels[train_index], labels[test_index]

            # Train the model
            model.fit(X_train, y_train)

            # Evaluate the model
            predictions = model.predict(X_test)
            y_pred = predictions
            y_test_encoded = y_test

            # Calculate evaluation metrics for the current fold
            accuracy = np.mean(y_pred == y_test_encoded)
            sensitivity = recall_score(y_test, y_pred, pos_label=1)
            specificity = recall_score(y_test, y_pred, pos_label=0)
            auc_roc = roc_auc_score(y_test, predictions)
            mcc = matthews_corrcoef(y_test, y_pred)
            precision = precision_score(y_test, y_pred, pos_label=1)
            f1 = f1_score(y_test, y_pred, pos_label=1)
            # Append metrics to lists
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            auc_roc_list.append(auc_roc)
            mcc_list.append(mcc)
            precision_list.append(precision)
            f1_list.append(f1)

            # Print metrics for the current fold
            print(f"Accuracy: {accuracy}")
            print(f"Sensitivity: {sensitivity}")
            print(f"Specificity: {specificity}")
            print(f"AUC-ROC: {auc_roc}")
            print(f"MCC: {mcc}")
            print(f"Precision: {precision}")
            print(f"F1 Score: {f1}")
            gc.collect()

        # Calculate average metrics
        average_accuracy = np.mean(accuracy_list)
        average_sensitivity = np.mean(sensitivity_list)
        average_specificity = np.mean(specificity_list)
        average_auc_roc = np.mean(auc_roc_list)
        average_mcc = np.mean(mcc_list)
        average_precision = np.mean(precision_list)
        average_f1 = np.mean(f1_list)

        time_end = time.time()
        memory_end = psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2
        

    # Append results to DataFrame
    results_df.loc[len(results_df)] = [name, average_accuracy, average_sensitivity, average_specificity,
                                       average_auc_roc, average_mcc, average_precision, average_f1,
                                       memory_end - memory_start, time_end - time_start]





In [14]:
from sklearn.linear_model import LogisticRegression
# SVM
lr_model = LogisticRegression(n_jobs=-1, random_state=0)
evaluate_model(lr_model, 'LR', data, labels)




[INFO] Fold 1 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9448476052249637
Sensitivity: 0.9499274310595065
Specificity: 0.9397677793904209
AUC-ROC: 0.9448476052249637
MCC: 0.8897411305171522
Precision: 0.9403735632183908
F1 Score: 0.9451263537906137

[INFO] Fold 2 / 10 for LR


/opt/tljh/user/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.9499274310595065
Sensitivity: 0.951378809869376
Specificity: 0.9484760522496372
AUC-ROC: 0.9499274310595066
MCC: 0.8998586532323145
Precision: 0.9486251808972503
F1 Score: 0.95

[INFO] Fold 3 / 10 for LR
Accuracy: 0.941944847605225
Sensitivity: 0.9470246734397678
Specificity: 0.9368650217706821
AUC-ROC: 0.941944847605225
MCC: 0.8839353156361267
Precision: 0.9375
F1 Score: 0.9422382671480145

[INFO] Fold 4 / 10 for LR
Accuracy: 0.9531930333817126
Sensitivity: 0.9477503628447025
Specificity: 0.9586357039187228
AUC-ROC: 0.9531930333817126
MCC: 0.9064397706732665
Precision: 0.9581804842259721
F1 Score: 0.9529368843487779

[INFO] Fold 5 / 10 for LR
Accuracy: 0.941944847605225
Sensitivity: 0.9390420899854862
Specificity: 0.9448476052249637
AUC-ROC: 0.941944847605225
MCC: 0.8839045908993081
Precision: 0.9445255474452555
F1 Score: 0.9417758369723436

[INFO] Fold 6 / 10 for LR
Accuracy: 0.941944847605225
Sensitivity: 0.9462989840348331
Specificity: 0.9375907111756169
AUC-ROC: 0.9419

In [15]:
#NB
nb_model = GaussianNB()
evaluate_model(nb_model, 'NB', data, labels)


[INFO] Fold 1 / 10 for NB
Accuracy: 0.8447024673439768
Sensitivity: 0.888243831640058
Specificity: 0.8011611030478955
AUC-ROC: 0.8447024673439768
MCC: 0.6920339138758796
Precision: 0.8170894526034713
F1 Score: 0.851182197496523

[INFO] Fold 2 / 10 for NB
Accuracy: 0.8421625544267054
Sensitivity: 0.8969521044992743
Specificity: 0.7873730043541364
AUC-ROC: 0.8421625544267054
MCC: 0.6884710272399877
Precision: 0.8083714846304775
F1 Score: 0.8503611971104231

[INFO] Fold 3 / 10 for NB
Accuracy: 0.8457910014513788
Sensitivity: 0.9085631349782293
Specificity: 0.7830188679245284
AUC-ROC: 0.8457910014513788
MCC: 0.6970974246338738
Precision: 0.8072211476466795
F1 Score: 0.8548992830317514

[INFO] Fold 4 / 10 for NB
Accuracy: 0.8370827285921626
Sensitivity: 0.8788098693759071
Specificity: 0.795355587808418
AUC-ROC: 0.8370827285921626
MCC: 0.6765254438060284
Precision: 0.8111185532484929
F1 Score: 0.8436084987809126

[INFO] Fold 5 / 10 for NB
Accuracy: 0.8421625544267054
Sensitivity: 0.88751814

In [16]:

# KNN
knn_model = KNeighborsClassifier()
evaluate_model(knn_model, 'KNN', data, labels)




[INFO] Fold 1 / 10 for KNN
Accuracy: 0.6770682148040639
Sensitivity: 0.9891146589259797
Specificity: 0.36502177068214803
AUC-ROC: 0.6770682148040639
MCC: 0.4532365249122144
Precision: 0.6090259159964254
F1 Score: 0.7538716814159292

[INFO] Fold 2 / 10 for KNN
Accuracy: 0.6846879535558781
Sensitivity: 0.9927431059506531
Specificity: 0.37663280116110304
AUC-ROC: 0.6846879535558781
MCC: 0.46895356076019057
Precision: 0.614279299506062
F1 Score: 0.7589459084604716

[INFO] Fold 3 / 10 for KNN
Accuracy: 0.6883164005805515
Sensitivity: 0.9898403483309144
Specificity: 0.3867924528301887
AUC-ROC: 0.6883164005805515
MCC: 0.4721454803742077
Precision: 0.6174739701222273
F1 Score: 0.7605241148592138

[INFO] Fold 4 / 10 for KNN
Accuracy: 0.693033381712627
Sensitivity: 0.9905660377358491
Specificity: 0.3955007256894049
AUC-ROC: 0.6930333817126271
MCC: 0.4803753222194861
Precision: 0.6210191082802548
F1 Score: 0.7634228187919463

[INFO] Fold 5 / 10 for KNN
Accuracy: 0.6756168359941945
Sensitivity: 0

In [17]:
# Random Forest
rf_model = RandomForestClassifier(random_state=0, n_jobs=-1)
evaluate_model(rf_model, 'Random Forest', data, labels)



[INFO] Fold 1 / 10 for Random Forest
Accuracy: 0.9350507982583455
Sensitivity: 0.9100145137880987
Specificity: 0.9600870827285921
AUC-ROC: 0.9350507982583454
MCC: 0.8711944383816782
Precision: 0.957983193277311
F1 Score: 0.9333829549683663

[INFO] Fold 2 / 10 for Random Forest
Accuracy: 0.9397677793904209
Sensitivity: 0.918722786647315
Specificity: 0.9608127721335269
AUC-ROC: 0.9397677793904211
MCC: 0.8803156734851084
Precision: 0.9590909090909091
F1 Score: 0.938472942920682

[INFO] Fold 3 / 10 for Random Forest
Accuracy: 0.941944847605225
Sensitivity: 0.9281567489114659
Specificity: 0.9557329462989841
AUC-ROC: 0.9419448476052251
MCC: 0.8842259624919135
Precision: 0.9544776119402985
F1 Score: 0.9411331861662987

[INFO] Fold 4 / 10 for Random Forest
Accuracy: 0.9433962264150944
Sensitivity: 0.9216255442670537
Specificity: 0.965166908563135
AUC-ROC: 0.9433962264150945
MCC: 0.8876342628942557
Precision: 0.9635811836115327
F1 Score: 0.9421364985163204

[INFO] Fold 5 / 10 for Random Forest

In [18]:
import lightgbm
# LGBM
lgb_model = lightgbm.LGBMClassifier(random_state=0, n_jobs = -1)
evaluate_model(lgb_model, 'LGBM', data, labels)


[INFO] Fold 1 / 10 for LGBM
[LightGBM] [Info] Number of positive: 12401, number of negative: 12401
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 6.945747 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3571326
[LightGBM] [Info] Number of data points in the train set: 24802, number of used features: 20789
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Accuracy: 0.956821480406386
Sensitivity: 0.9644412191582002
Specificity: 0.9492017416545718
AUC-ROC: 0.9568214804063861
MCC: 0.9137490722813542
Precision: 0.949964260185847
F1 Score: 0.9571480014404032

[INFO] Fold 2 / 10 for LGBM
[LightGBM] [Info] Number of positive: 12401, number of negative: 12401
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 7.113055 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3570883
[LightGBM] [Info] Number 

In [19]:
# Save results to a CSV file
results_df.to_csv('DS1_VGG16.csv', index=False)

In [20]:
results_df

,Model,Average Accuracy,Average Sensitivity,Average Specificity,Average AUC-ROC,Average MCC,Average Precision,Average F1 Score,Memory Used (MB),Time (s)
0,LR,0.944299,0.945497,0.943102,0.944299,0.888635,0.943290,0.944374,-1564.152344,2406.953203
1,NB,0.839937,0.887800,0.792074,0.839937,0.683099,0.810276,0.847228,2374.156250,132.971478
2,KNN,0.683504,0.989767,0.377240,0.683504,0.464287,0.613826,0.757721,2339.183594,354.690863
3,Random Forest,0.939183,0.916468,0.961898,0.939183,0.879326,0.960106,0.937747,2407.542969,586.537022
4,LGBM,0.952138,0.954061,0.950214,0.952137,0.904334,0.950430,0.952215,2948.218750,2577.259463
